In [8]:
import pandas as pd
import os
import datetime

In [15]:
# Load preferred sailings per month (First Data)
preferred_sailing_df = pd.read_excel('ShipsPerRegion_V2.xlsx', sheet_name = 'Main Data')

In [16]:
preferred_sailing_df.head()

,LOB,Prefered Sailing pm,Starting Region,Ending Region,Avg Voyage days,Med Voyage days,Min Voyage days,Max Voyage days,Distinct_Voyage_Count,Voyage Frequency per month,Total Ships Needed per Route,Total Ships Needed per Route (Round),Acceptable classes,Acceptable ships
0,BAJ,1.0,South America,Asia,57.081081,58.0,31,81,37,0.525568,1.902703,2.0,"['J30', 'J32', 'J33']",NaN
1,ESA-S,1.0,Continental Europe,Middle East,56.666667,59.0,27,86,15,0.529412,1.888889,2.0,"['C33', 'J32', 'C30']",NaN
2,EXP-A,3.0,Continental Europe,Middle East,42.329897,42.0,14,73,97,0.708719,4.232990,4.0,"['C33', 'N37B', 'N43', 'J30']",NaN
3,GIP,1.0,North America,Middle East,75.641026,77.0,31,115,39,0.396610,2.521368,2.5,"['J33', 'N37A', 'N43', 'C33']",NaN
4,GIR-C,3.0,Middle East,Continental Europe,69.835821,67.0,37,114,67,0.429579,6.983582,7.0,"['C33', 'N37B', 'N43']",NaN


In [17]:
# Load actual ships and their current regions (Second Data from Dim Voyage)
ship_availability_df = pd.read_excel('ShipsPerRegion_V2.xlsx', sheet_name = 'Commence Region') 
# Filter for MonthYear = '2025-01'
ship_availability_df = ship_availability_df[ship_availability_df['MonthYear'] == '2025-01']

In [18]:
ship_availability_df.head()

,vesselcode,Class,MonthYear,lineofbusinesscode,CommencingPort,region,MappedRegion
0,BBRIS,C25,2025-01,TAE,HOUSTON,USWG,North America
4,BCASA,C25,2025-01,AMER TRAMP,SANTA ROSALIA,MEXICO,South America
7,BHSTN,C25,2025-01,AAG,MAP TA PHUT,FAR EAST,Asia
11,BNOLA,C25,2025-01,PACS,GEBZE,MEDITERRANEAN,Middle East
18,BOOSL,J32,2025-01,GIR-U,FUJAIRAH,PERSIAN GULF,Middle East


In [19]:
import pandas as pd

seen_ships = set()  # Track all ships ever seen

def allocate_ships(preferred_sailing_df, ship_availability_df, months):
    ship_status = {}  # Stores latest end regions and return months
    unassigned_ships = set()  # Ships not allocated in previous months
    all_allocations = []  # Store final allocation output

    for month in months:
        print(f"\n🔹 Allocating ships for {month}...")

        # 🚢 **Step 1: Refresh Demand**
        lob_demand = preferred_sailing_df.set_index(['LOB', 'Starting Region', 'Ending Region'])['Prefered Sailing pm'].to_dict()
        print(f"🔍 Initial Demand: {lob_demand}")

        # 🚢 **Step 2: Get Available Ships for the Month**
        available_ships_now = set(ship_availability_df[ship_availability_df['MonthYear'] == month]['vesselcode'])
        new_ships = available_ships_now - seen_ships  # Ships appearing for the first time
        seen_ships.update(available_ships_now)

        # Identify ships returning from previous assignments
        returned_ships = {ship for ship, details in ship_status.items() if details['return_month'] == month}

        # Combine Available Ships
        available_ships = new_ships | returned_ships | unassigned_ships

        # Remove Busy Ships (those returning later)
        busy_ships = {ship for ship, details in ship_status.items() if details['return_month'] > month}
        available_ships -= busy_ships

        # Convert to List with Correct Start Regions
        available_ships_list = []
        for ship in available_ships:
            if ship in ship_status:  # If previously assigned, start from its last ending region
                start_region = ship_status[ship]['end_region']
            else:  # If new ship, get from availability data
                start_region = ship_availability_df.loc[ship_availability_df['vesselcode'] == ship, 'MappedRegion'].values[0]
            available_ships_list.append((ship, start_region))

        print(f"🚢 Available Ships for {month}: {available_ships_list}")

        # 🚢 **Step 3: Assign Ships Based on Demand**
        allocated_ships = []
        remaining_unassigned_ships = set(ship for ship, _ in available_ships_list)
        sorted_lob_keys = sorted(lob_demand.keys(), key=lambda k: lob_demand[k], reverse=True)

        for ship_name, ship_region in available_ships_list:
            assigned_lob = next((key for key in sorted_lob_keys if key[1] == ship_region and lob_demand.get(key, 0) > 0), None)
            
            if assigned_lob:
                allocated_ships.append({
                    'Month': month,
                    'vesselcode': ship_name,
                    'Assigned_LOB': assigned_lob[0],
                    'Starting Region': assigned_lob[1],
                    'Ending Region': assigned_lob[2]
                })
                lob_demand[assigned_lob] -= 1  # Reduce demand
                remaining_unassigned_ships.discard(ship_name)

                # Get voyage duration and set new return month
                voyage_days = preferred_sailing_df[
                    (preferred_sailing_df['LOB'] == assigned_lob[0]) & 
                    (preferred_sailing_df['Starting Region'] == assigned_lob[1]) & 
                    (preferred_sailing_df['Ending Region'] == assigned_lob[2])
                ]['Avg Voyage days']

                avg_voyage_days = int(voyage_days.values[0]) if not voyage_days.empty else 60  # Default to 60 days if missing

                next_available_month_index = min(months.index(month) + (avg_voyage_days // 30), len(months) - 1)
                next_available_month = months[next_available_month_index]

                # Update ship's last known region and return time
                ship_status[ship_name] = {'end_region': assigned_lob[2], 'return_month': next_available_month}

        # 🚢 **Step 4: Update Unassigned Ships List**
        unassigned_ships = remaining_unassigned_ships  

        # 🚢 **Step 5: Print Summary**
        print(f"✅ Total available ships in {month}: {len(available_ships)}")
        print(f"🚢 Allocated Ships in {month}: {len(allocated_ships)}")
        print(f"❌ Unassigned Ships in {month}: {len(unassigned_ships)}")
        print(f"📉 LOB demand remaining after allocation in {month}: {sum(lob_demand.values())}")

        all_allocations.extend(allocated_ships)

    allocation_df = pd.DataFrame(all_allocations)
    print("\n📝 Final Allocation Output:")
    print(allocation_df)
    return allocation_df


In [20]:
months = ['2025-01', '2025-02', '2025-03', '2025-04', '2025-05', '2025-06']
allocation_result = allocate_ships(preferred_sailing_df, ship_availability_df, months)
# Generate timestamp for unique filenames
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Save the file with a timestamp
filename = f"Ship_Allocation_Till_June_{timestamp}.xlsx"
allocation_result.to_excel(filename, index=False)

print(f"✅ Allocation saved as: {filename}")



🔹 Allocating ships for 2025-01...
🔍 Initial Demand: {('BAJ', 'South America', 'Asia'): 1.0, ('ESA-S', 'Continental Europe', 'Middle East'): 1.0, ('EXP-A', 'Continental Europe', 'Middle East'): 3.0, ('GIP', 'North America', 'Middle East'): 1.0, ('GIR-C', 'Middle East', 'Continental Europe'): 3.0, ('GIR-M', 'Middle East', 'Mediterranean'): 1.0, ('GIR-U', 'Middle East', 'North America'): 1.0, ('HBR-C', 'Asia', 'Continental Europe'): 2.0, ('HBR-U Panama', 'Asia', 'North America'): 1.0, ('HBR-U Suez', 'Asia', 'North America'): 2.0, ('JAB', 'Asia', 'South America'): 0.5, ('LAS-EN', 'South America', 'North America'): 2.0, ('LAS-ES', 'North America', 'South America'): 2.0, ('PACS', 'Continental Europe', 'Asia'): 1.0, ('TAE', 'North America', 'Continental Europe'): 2.0, ('TAW', 'Continental Europe', 'North America'): 2.0, ('TPW', 'North America', 'Asia'): 3.0, ('UMR-E', 'North America', 'Mediterranean'): 1.0, ('UMR-W', 'Mediterranean', 'North America'): 1.0, ('AAG', 'Asia', 'Middle East'): 1.0

##next steps
1. Compare with actuals for 3 months
2. Add constraints - which ships are allowed for which TL
3. Tonnage instead of preferred sailing per month


Constraints to Add to Your Greedy Algorithm
1️⃣ Single Assignment Per Month → Each ship can be assigned to only one LOB per month.

2️⃣ LOB Demand Limit → Do not assign more ships than the available demand for a given LOB in a month.

3️⃣ Starting Region Consistency → If a ship is reallocated, its next LOB must start from its last ending region.

4️⃣ Voyage Duration Constraint → A ship returns only after completing its voyage (based on Avg Voyage days).

5️⃣ Availability Constraint → Only ships available in the current month can be assigned.

6️⃣ Carry Forward Unassigned Ships → If a ship is unassigned in a month, it should be available in the next month.


In [75]:
import pandas as pd  

# Load the Excel file
acceptable_classes_df = pd.read_excel('ShipsPerRegion_V2.xlsx', sheet_name='Acceptable Classes')

# Convert "Class" column from CSV-like string to list
acceptable_classes_df['Class'] = acceptable_classes_df['Class'].astype(str).apply(lambda x: [cls.strip() for cls in x.split(",")] if "," in x else [x.strip()])

# Verify the output
print(acceptable_classes_df['Class'].head(10))
print(acceptable_classes_df['Class'].apply(type).value_counts())  # Should show <class 'list'>

acceptable_classes = acceptable_classes_df.set_index('LOB')['Class'].to_dict()
print(acceptable_classes)  # Should now work without errors



0                             [C25, J25]
1                        [C25, J25, J30]
2                        [J30, J32, J33]
3                        [C33, J32, C30]
4                  [C33, N37B, N43, J30]
5                  [J33, N37A, N43, C33]
6                       [C33, N37B, N43]
7                  [C33, J33, N37A, N43]
8    [C25, C26, C30, C33, J30, J32, J33]
9                        [C33, C38, J25]
Name: Class, dtype: object
Class
<class 'list'>    26
Name: count, dtype: int64
{'AAG': ['C25', 'J25'], 'AGE': ['C25', 'J25', 'J30'], 'BAJ': ['J30', 'J32', 'J33'], 'ESA-S': ['C33', 'J32', 'C30'], 'EXP-A': ['C33', 'N37B', 'N43', 'J30'], 'GIP': ['J33', 'N37A', 'N43', 'C33'], 'GIR-C': ['C33', 'N37B', 'N43'], 'GIR-U': ['C33', 'J33', 'N37A', 'N43'], 'HBR-C': ['C25', 'C26', 'C30', 'C33', 'J30', 'J32', 'J33'], 'HBR-U Panama': ['C33', 'C38', 'J25'], 'HBR-U Suez': ['C33', 'C38', 'D37', 'F37', 'J32', 'C30'], 'JAB': ['C25', 'J30'], 'JTG742': ['J32'], 'JTG743': ['N37B'], 'LAS-EN': ['C25', 'J25',

In [76]:
import pandas as pd
import ast  # To safely convert string lists to Python lists

seen_ships = set()  # Track all ships ever seen

def allocate_ships(preferred_sailing_df, ship_availability_df, acceptable_classes_df, months):
    ship_status = {}  # Stores latest end regions and return months
    unassigned_ships = set()  # Ships not allocated in previous months
    all_allocations = []  # Store final allocation output


    for month in months:
        print(f"\n🔹 Allocating ships for {month}...")

        # 🚢 **Step 1: Refresh Demand**
        lob_demand = preferred_sailing_df.set_index(['LOB', 'Starting Region', 'Ending Region'])['Prefered Sailing pm'].to_dict()
        print(f"🔍 Initial Demand: {lob_demand}")

        # 🚢 **Step 2: Get Available Ships for the Month**
        available_ships_now = set(ship_availability_df[ship_availability_df['MonthYear'] == month]['vesselcode'])
        new_ships = available_ships_now - seen_ships  # Ships appearing for the first time
        seen_ships.update(available_ships_now)

        # Identify ships returning from previous assignments
        returned_ships = {ship for ship, details in ship_status.items() if details['return_month'] == month}

        # Combine Available Ships
        available_ships = new_ships | returned_ships | unassigned_ships

        # Remove Busy Ships (those returning later)
        busy_ships = {ship for ship, details in ship_status.items() if details['return_month'] > month}
        available_ships -= busy_ships

        # Convert to List with Correct Start Regions
        available_ships_list = []
        for ship in available_ships:
            if ship in ship_status:  # If previously assigned, start from its last ending region
                start_region = ship_status[ship]['end_region']
            else:  # If new ship, get from availability data
                start_region = ship_availability_df.loc[ship_availability_df['vesselcode'] == ship, 'MappedRegion'].values[0]
            available_ships_list.append((ship, start_region))

        print(f"🚢 Available Ships for {month}: {available_ships_list}")

        # 🚢 **Step 3: Assign Ships Based on Demand & Acceptable Classes**
        allocated_ships = []
        remaining_unassigned_ships = set(ship for ship, _ in available_ships_list)
        sorted_lob_keys = sorted(lob_demand.keys(), key=lambda k: lob_demand[k], reverse=True)

        for ship_name, ship_region in available_ships_list:
            # Get ship class from availability data
            ship_class = ship_availability_df.loc[ship_availability_df['vesselcode'] == ship_name, 'Class'].values[0]

            # Find an eligible LOB that matches region + ship class constraint
            assigned_lob = next((
                key for key in sorted_lob_keys 
                if key[1] == ship_region and lob_demand.get(key, 0) > 0 and
                ship_class in acceptable_classes.get(key[0], [])  # Ensure ship class is allowed
            ), None)

            if assigned_lob:
                allocated_ships.append({
                    'Month': month,
                    'vesselcode': ship_name,
                    'Assigned_LOB': assigned_lob[0],
                    'Starting Region': assigned_lob[1],
                    'Ending Region': assigned_lob[2]
                })
                lob_demand[assigned_lob] -= 1  # Reduce demand
                remaining_unassigned_ships.discard(ship_name)

                # Get voyage duration and set new return month
                voyage_days = preferred_sailing_df[
                    (preferred_sailing_df['LOB'] == assigned_lob[0]) & 
                    (preferred_sailing_df['Starting Region'] == assigned_lob[1]) & 
                    (preferred_sailing_df['Ending Region'] == assigned_lob[2])
                ]['Avg Voyage days']

                avg_voyage_days = int(voyage_days.values[0]) if not voyage_days.empty else 60  # Default to 60 days if missing

                next_available_month_index = min(months.index(month) + (avg_voyage_days // 30), len(months) - 1)
                next_available_month = months[next_available_month_index]

                # Update ship's last known region and return time
                ship_status[ship_name] = {'end_region': assigned_lob[2], 'return_month': next_available_month}

        # 🚢 **Step 4: Update Unassigned Ships List**
        unassigned_ships = remaining_unassigned_ships  

        # 🚢 **Step 5: Print Summary**
        print(f"✅ Total available ships in {month}: {len(available_ships)}")
        print(f"🚢 Allocated Ships in {month}: {len(allocated_ships)}")
        print(f"❌ Unassigned Ships in {month}: {len(unassigned_ships)}")
        print(f"📉 LOB demand remaining after allocation in {month}: {sum(lob_demand.values())}")

        all_allocations.extend(allocated_ships)

    allocation_df = pd.DataFrame(all_allocations)
    print("\n📝 Final Allocation Output:")
    print(allocation_df)  
    return allocation_df


In [77]:
# ✅ Corrected function call with all three required dataframes
months = ['2025-01', '2025-02', '2025-03', '2025-04', '2025-05', '2025-06']
allocation_result = allocate_ships(preferred_sailing_df, ship_availability_df, acceptable_classes_df, months)

# ✅ Generate timestamp for unique filenames
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# ✅ Save the file with a timestamp
filename = f"Ship_Allocation_Till_June_{timestamp}.xlsx"
allocation_result.to_excel(filename, index=False)

print(f"✅ Allocation saved as: {filename}")


🔹 Allocating ships for 2025-01...
🔍 Initial Demand: {('BAJ', 'South America', 'Asia'): 1.0, ('ESA-S', 'Continental Europe', 'Middle East'): 1.0, ('EXP-A', 'Continental Europe', 'Middle East'): 3.0, ('GIP', 'North America', 'Middle East'): 1.0, ('GIR-C', 'Middle East', 'Continental Europe'): 3.0, ('GIR-M', 'Middle East', 'Mediterranean'): 1.0, ('GIR-U', 'Middle East', 'North America'): 1.0, ('HBR-C', 'Asia', 'Continental Europe'): 2.0, ('HBR-U Panama', 'Asia', 'North America'): 1.0, ('HBR-U Suez', 'Asia', 'North America'): 2.0, ('JAB', 'Asia', 'South America'): 0.5, ('LAS-EN', 'South America', 'North America'): 2.0, ('LAS-ES', 'North America', 'South America'): 2.0, ('PACS', 'Continental Europe', 'Asia'): 1.0, ('TAE', 'North America', 'Continental Europe'): 2.0, ('TAW', 'Continental Europe', 'North America'): 2.0, ('TPW', 'North America', 'Asia'): 3.0, ('UMR-E', 'North America', 'Mediterranean'): 1.0, ('UMR-W', 'Mediterranean', 'North America'): 1.0, ('AAG', 'Asia', 'Middle East'): 1.0